# Floor Plan Generator

This notebook provides an interface to generate floor plans. You can input a text prompt, choose the number of images to generate, select the inference steps, and pick the scheduler.


### Imports

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display
from PIL import Image
import ipywidgets as widgets

### Image generation funtion

In [ ]:
def generate_images(prompt, num_images, scheduler, steps):
    pipeline = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

    # Load the LoRA weights for the model
    pipeline.load_lora_weights("Checkpoint_L1/checkpoint-250", weight_name="pytorch_lora_weights.safetensors")

    # Set the scheduler
    if scheduler == "DDIM":
        pipeline.scheduler = pipeline.scheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler_ddim")
    # Add other schedulers as needed

    # Generate images based on the prompt
    images = []
    for _ in range(num_images):
        image = pipeline(prompt, num_inference_steps=steps, guidance_scale=7.5).images[0]
        images.append(image)
    return images


### Inputs

In [ ]:
# Textbox for prompt
prompt_widget = widgets.Textarea(
    value='',
    placeholder='Enter your prompt...',
    description='Prompt:',
    disabled=False
)

# Slider for number of images
num_images_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=4,
    step=1,
    description='Num Images:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Slider for inference steps
steps_widget = widgets.IntSlider(
    value=50,
    min=1,
    max=100,
    step=1,
    description='Inference Steps:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Dropdown for scheduler
scheduler_widget = widgets.Dropdown(
    options=["DDIM", "PNDM", "K_EULER", "DPMSolverMultistep", "HeunDiscrete", "K_EULER_ANCESTRAL"],
    value='DDIM',
    description='Scheduler:',
    disabled=False,
)

# Display widgets
display(prompt_widget, num_images_widget, steps_widget, scheduler_widget)


## Generate and display images

In [ ]:
# Button to generate images
generate_button = widgets.Button(
    description='Generate Images',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate images',
    icon='check'
)

# Output widget to display images
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with output:
        output.clear_output()
        prompt = prompt_widget.value
        num_images = num_images_widget.value
        steps = steps_widget.value
        scheduler = scheduler_widget.value
        
        images = generate_images(prompt, num_images, scheduler, steps)
        for idx, img in enumerate(images):
            display(img)
            img.save(f"generated_image_{idx+1}.png")

generate_button.on_click(on_button_click)

# Display button and output
display(generate_button, output)

